In [1]:
import numpy as np

from GS_functions import *
from cv2 import imwrite
from modeling.models.bethge import BethgeModel
from analysis.Stat_utils import get_site_corr
from scripts.pattern_stimulus_test import get_label_pattern_stimuli
def img2matrix(imgmainpath,suffix='.png'):
    '''
    make sure all imgs are the same size
    '''
    pathlist=GF.filelist_suffix(imgmainpath,suffix)
    im_shape=np.array(PIL.Image.open(f"{imgmainpath}/{pathlist[0]}").convert('L')).shape[0]
    im_matrix=np.zeros((len(pathlist),im_shape,im_shape))
    for i in range(len(pathlist)):
        # print('Change img:',i+1)
        im_matrix[i,:,:]= np.array(PIL.Image.open(f"{imgmainpath}/{pathlist[i]}").convert('L'))

    return im_matrix


In [2]:
data_partial_1 = np.load('D:/school/research/CNN_Tang_project/data/Processed_pattern_stimuli/crop_100_resize_1.npy')
data_partial_2 = np.load('D:/school/research/CNN_Tang_project/data/Processed_pattern_stimuli/crop_100_resize_2.npy')
data_partial_3 = np.load('D:/school/research/CNN_Tang_project/data/Processed_pattern_stimuli/crop_100_resize_3.npy')
data_partial_4 = np.load('D:/school/research/CNN_Tang_project/data/Processed_pattern_stimuli/crop_100_resize_4.npy')

PS_imgs_all = np.concatenate((data_partial_1, data_partial_2, data_partial_3, data_partial_4), 0)
PS_labels = np.load('D:/school/research/CNN_Tang_project/data/Processed_pattern_stimuli/labels.npy')
PS_labels_all = np.concatenate((PS_labels, PS_labels, PS_labels, PS_labels), 0)

sites = [ 'm1s1', 'm1s2', 'm1s3','m2s1', 'm2s2', 'm3s1']
all_matrix = []
full_dict = {}
for site in sites:
    val_y = np.load('../data/Processed_Tang_data/all_sites_data_prepared/New_response_data/valRsp_' + site + '.npy')
    channels = 256
    num_layers = 9
    input_size = 50
    output_size = val_y.shape[1]
    first_k = 9
    later_k = 3
    pool_size = 2
    factorized = True
    num_maps = 1

    dir = 'D:/school/research/CNN_Tang_project/analysis/Visualization/Shared_Core_avg_border_vis/' + site + '/'
    matrix = img2matrix(dir,'.jpg')
    net = BethgeModel(channels=channels, num_layers=num_layers, input_size=input_size,
                      output_size=output_size, first_k=first_k, later_k=later_k,
                      input_channels=1, pool_size=pool_size, factorized=True,
                      num_maps=num_maps).cuda()

    all_corr = []
    all_iteration_label = []
    for i in range(10):
        net.load_state_dict(torch.load(f'../saved_models/Sample_size_test_models/{i}_10_{site}_shared_core_256_9'))
        corr = get_site_corr(net,'cuda',site)
        all_corr.append(corr)

        all_neuron_labels, top_indexes, top_rsp = get_label_pattern_stimuli(PS_imgs_all, PS_labels_all, 50, 'cuda', net, stacked_data=True)
        all_iteration_label.append(all_neuron_labels)

    all_iteration_label = np.array(all_iteration_label)
    all_corr = np.array(all_corr)



    site_dict = {'corr': all_corr, 'vis': matrix, 'label':all_iteration_label}
    #site_dict = {'corr': all_corr}
    full_dict[site] = site_dict
np.save("cosyne_dict_2", full_dict)


There are  302  files in this directory


100%|██████████| 10/10 [00:03<00:00,  3.18it/s]


There are  330  files in this directory


100%|██████████| 10/10 [00:03<00:00,  2.93it/s]


There are  175  files in this directory


...

100%|██████████| 10/10 [00:03<00:00,  3.16it/s]


There are  259  files in this directory


100%|██████████| 10/10 [00:02<00:00,  3.64it/s]


There are  324  files in this directory


100%|██████████| 10/10 [00:03<00:00,  2.97it/s]
